<a href="https://colab.research.google.com/github/JannisWolf/evaluating-edge-accelerator/blob/main/model_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as functional
import torch.optim as optim

import numpy as np

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
!ls '/content/gdrive/My Drive/data/autoencoder'

model.onnx  net-state-3072-scaled2-standard


In [6]:
inputSize = 3*1024

In [7]:
# auto encoder class

class GrindNet(nn.Module):
  def __init__(self, **kwargs):
    super().__init__()
    sizes = [inputSize, 256]
    self.layers = nn.ModuleList()

    for size1, size2 in zip(sizes, sizes[1:]):
      self.layers.append(nn.Linear(in_features=size1, out_features=size2))
    for size1, size2 in zip(reversed(sizes), list(reversed(sizes))[1:]):
      self.layers.append(nn.Linear(in_features=size1, out_features=size2))
  
  def forward(self, x):
    for layer in self.layers:
      x = layer(x)
      x = functional.leaky_relu(x)
    return x

In [8]:
model = GrindNet()
path = F"/content/gdrive/My Drive/data/autoencoder/"
model_name = "net-state-3072-scaled2-standard"

In [9]:
model.load_state_dict(torch.load(path+model_name))
model.eval()

GrindNet(
  (layers): ModuleList(
    (0): Linear(in_features=3072, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=3072, bias=True)
  )
)

In [10]:
with torch.no_grad():
     torch.onnx.export(model,
     torch.randn(*(inputSize,)),
     path + "/model.onnx",
     export_params=True,
     opset_version=10,
     do_constant_folding=True,
     input_names=['input'],
     output_names=['output'])

In [ ]:
!pip install onnx==1.8.1 onnx2keras pytorch2keras

In [29]:
import onnx
from onnx2keras import onnx_to_keras
from pytorch2keras.converter import pytorch_to_keras
from torch.autograd import Variable
import tensorflow as tf

In [128]:
input_np = np.random.uniform(0, 1, (3072)).astype('float32')
a = Variable(torch.FloatTensor(input_np))
print(input_np)
print(a)

# a = torch.rand(3072)

[0.11086931 0.87515277 0.27407765 ... 0.2219126  0.615592   0.84407085]
tensor([0.1109, 0.8752, 0.2741,  ..., 0.2219, 0.6156, 0.8441])


In [ ]:
k_model = pytorch_to_keras(model, a,input_shapes=[(3072,)], verbose=True)

In [33]:
k_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_0 (InputLayer)            [(None, 3072)]       0                                            
__________________________________________________________________________________________________
7_const1 (Lambda)               (256,)               0           input_0[0][0]                    
__________________________________________________________________________________________________
6 (Dense)                       (None, 256)          786432      input_0[0][0]                    
__________________________________________________________________________________________________
7 (Lambda)                      (None, 256)          0           7_const1[0][0]                   
                                                                 6[0][0]                    

In [34]:
print(model)

GrindNet(
  (layers): ModuleList(
    (0): Linear(in_features=3072, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=3072, bias=True)
  )
)


In [138]:
# convert pytorch tensor to tf tensor
a_tf = tf.convert_to_tensor([input_np]) 

# run inference on the model
p = model(a)
k = k_model.predict(a_tf)

# print results
print("Input Pytorch {}".format(a))
print("Input Tensorflow tensor({})".format(a_tf[0]))
print("Output Pytorch {}".format(p))
print("Output Tensorflow tensor({})".format(k[0]))

# equal function as the precision differs
def equal(l1, l2, p=False):
  '''
  Checks to which precision it is equal
  '''
  diff = abs(l1 - l2)
  max_diff = np.max(diff)
  if p:
    print("Maximum difference is {}".format(max_diff))
  for i in range(10):
    prec = 10**-i
    if max_diff > prec:
      p = i
      break
  return "Equal until 10^-{}.".format(p)

# check if the values are the same
print(equal(k,p.detach().numpy()))

Input Pytorch tensor([0.1109, 0.8752, 0.2741,  ..., 0.2219, 0.6156, 0.8441])
Input Tensorflow tensor([0.11086931 0.87515277 0.27407765 ... 0.2219126  0.615592   0.84407085])
Output Pytorch tensor([ 2.6314,  2.3040,  3.3686,  ..., -0.0494,  0.0215,  2.4654],
       grad_fn=<LeakyReluBackward0>)
Output Tensorflow tensor([ 2.6314406   2.304026    3.368564   ... -0.04941374  0.02154568
  2.4654353 ])
Equal until 10^-5.


In [107]:
# prerequisites of the tensorflow lite inference
SAVED_MODEL_PATH = '/content/gdrive/My Drive/data/saved_models/test_variable'
TFLITE_FILE_PATH = '/content/gdrive/My Drive/data/test_variable.tflite'

tf.saved_model.save(
    k_model, SAVED_MODEL_PATH)

#converter = tf.lite.TFLiteConverter.from_keras_model(k_model)
converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_PATH)
tflite_model = converter.convert()

with open(TFLITE_FILE_PATH, 'wb') as f:
  f.write(tflite_model)

Tensor("model_4/7_const1/Const:0", shape=(256,), dtype=float32) Tensor("model_4/6/MatMul:0", shape=(None, 256), dtype=float32)
Tensor("model_4/11_const1/Const:0", shape=(3072,), dtype=float32) Tensor("model_4/10/MatMul:0", shape=(None, 3072), dtype=float32)
Tensor("inputs:0", shape=(256,), dtype=float32) Tensor("inputs_1:0", shape=(None, 256), dtype=float32)
Tensor("inputs:0", shape=(3072,), dtype=float32) Tensor("inputs_1:0", shape=(None, 3072), dtype=float32)
Tensor("inputs:0", shape=(3072,), dtype=float32) Tensor("inputs_1:0", shape=(None, 3072), dtype=float32)
Tensor("inputs:0", shape=(256,), dtype=float32) Tensor("inputs_1:0", shape=(None, 256), dtype=float32)
Tensor("7_const1/Const:0", shape=(256,), dtype=float32) Tensor("6/MatMul:0", shape=(None, 256), dtype=float32)
Tensor("11_const1/Const:0", shape=(3072,), dtype=float32) Tensor("10/MatMul:0", shape=(None, 3072), dtype=float32)
Tensor("7_const1/Const:0", shape=(256,), dtype=float32) Tensor("6/MatMul:0", shape=(None, 256), dtyp


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: /content/gdrive/My Drive/data/saved_models/test_variable/assets


INFO:tensorflow:Assets written to: /content/gdrive/My Drive/data/saved_models/test_variable/assets


In [130]:
# Load the TFLite model in TFLite Interpreter
interpreter = tf.lite.Interpreter(TFLITE_FILE_PATH)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], [input_np])

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[ 2.631441    2.3040261   3.3685653  ... -0.04941375  0.02154616
   2.4654405 ]]


In [143]:
print("Pytorch vs. Tensorflow")
print(equal(k,p.detach().numpy(), p=True) + '\n')
print("Pytorch vs. Tensorflow lite")
print(equal(output_data,p.detach().numpy(), p=True) + '\n')
print("Tensorflow vs Tensorflow lite")
print(equal(k, output_data, p=True) + '\n')

Pytorch vs. Tensorflow
Maximum difference is 1.1682510375976562e-05
Equal until 10^-5.

Pytorch vs. Tensorflow lite
Maximum difference is 1.2159347534179688e-05
Equal until 10^-5.

Tensorflow vs Tensorflow lite
Maximum difference is 1.1444091796875e-05
Equal until 10^-5.

